In [8]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import psycopg2
import configparser
import datetime
import nbimporter
from General import *
from Daily_etl import *
from SQL import *
from Connection_to_pgAdmin4 import *

In [9]:
#installation 
#by default you might not have yahoo finance/ library, or psycopg2, or  nbimporternbimporter so you may have to install one
'''pip install yfinance'''
'''pip install nbimporter'''
'''pip install psycopg2'''

'pip install psycopg2'

In [10]:
#*********************
#This code was developed to work on the missing data that was intialy not collected with other functions. 
#*********************


#Reference cell G
def fetch_missing_symbol_data(symbol):
    try:
        # Create a Ticker object for the given symbol
        stock = yf.Ticker(symbol)
        
        # Fetch historical data for the stock from Jan 1, 2005 until today
        hist = stock.history(start = datetime.strptime("2005-01-01", "%Y-%m-%d"), end= datetime.now().strftime('%Y-%m-%d'))
        
         # Check if the historical data DataFrame is empty (no data returned)
        if hist.empty:
            return pd.DataFrame()# Return an empty DataFrame if no data found
        
        # Reset the index of the DataFrame which contains the dates
        hist.reset_index(inplace=True)
        
         # Add a new column to the DataFrame with the symbol name
        hist['Symbol'] = symbol

        #the desired column order, 
        #Because I was receving lots of colums that were not important, therefor i only want the the specified columns.
        desired_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Symbol']
        return hist[desired_columns]
    
    # Print an error message with the symbol and the exception message
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        return pd.DataFrame()

In [16]:
#***** 
#Creatig function to fetch data since 2005 as a basline for the project.


#Reference cell A
def fetch_historical_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        hist = stock.history(start="2005-01-01", end=datetime.now().strftime('%Y-%m-%d'))
        if hist.empty:
            return pd.DataFrame()
        
        hist.reset_index(inplace=True)
        hist['Symbol'] = symbol

        #the desired column order, 
        #Because I was receving lots of colums that were not important, therefor i only want the the specified columns.
        desired_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Symbol']
        return hist[desired_columns]
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        return pd.DataFrame()

In [12]:
#*******
#Creatig function to fetch Information on copanies that are listed based on symbol.
#This is slightly different because it populates information on companies.


#Reference cell B
def fetch_info_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        info = stock.info

        selected_columns = [
            "address1", "city", "state", "zip", "country", "phone", "website",
            "industry", "sector", "sectorKey", "symbol", "longName",
            "fullTimeEmployees", "profitMargins", "earningsGrowth", "revenueGrowth",
            "marketCap", "fiftyTwoWeekHigh", "fiftyTwoWeekLow", "currency", "exchange"
        ]

        info_data = {column: info.get(column, "N/A") for column in selected_columns}
        return pd.DataFrame([info_data])

    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        return None

In [13]:
#*****************
#Generating date dimension

#start_date = datetime.date(2005, 1, 1)
#end_date = datetime.date(2008, 12, 31)


#Reference cell C
def generate_date_dimension(start_date, end_date):
    # Generating a date range because i want to get the date upto 2050-12-31
    date_range = pd.date_range(start=start_date, end=end_date, name='Date')
    # Create DataFrame
    date_dimension = pd.DataFrame(date_range)
    # the columns to concider here are below
    date_dimension['Year'] = date_dimension['Date'].dt.year
    date_dimension['Quarter'] = date_dimension['Date'].dt.quarter
    date_dimension['Month'] = date_dimension['Date'].dt.month
    date_dimension["Week"] = date_dimension["Date"].dt.strftime("%U")
    date_dimension['Day'] = date_dimension['Date'].dt.day
    date_dimension['DayOfWeek'] = date_dimension['Date'].dt.dayofweek
    date_dimension['IsWeekend'] = date_dimension['DayOfWeek'].isin([5, 6])
    date_dimension['IsHoliday'] = False 
    date_dimension['FiscalYear'] = date_dimension['Year']  #  based on fiscal year
    date_dimension['FiscalQuarter'] = date_dimension['Date'].dt.quarter  #  based on fiscal quarters
    date_dimension['FiscalMonth'] = date_dimension['Date'].dt.month  #  based on fiscal months
    return date_dimension


#to see the result of this just uncomment the print statement below

In [14]:
#start_date = datetime.date(2005, 1, 1)
#end_date = datetime.date(2008, 12, 31)
#generate_date_dimension(start_date, end_date)

#to see the result of this just uncomment the print statement and year variable